In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
feature_list = ['x','y','w0','w1','w2','w3','w4','w5','w6','day','month','year', 'accuracy']
for i in range(0,24):
    feature_list.append('h' + str(i))

In [3]:
def prepare_data(df):    
    #Feature engineering
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-02T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    #df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    for i in range(0,24):
        df['h' + str(i)] = (((d_times.hour+ d_times.minute/60) + i) % 24) * fw[2]
    
    df['w0'] = ((d_times.weekday + 0) % 7) * fw[3]
    df['w1'] = ((d_times.weekday + 1) % 7) * fw[3]
    df['w2'] = ((d_times.weekday + 2) % 7) * fw[3]
    df['w3'] = ((d_times.weekday + 3) % 7) * fw[3]
    df['w4'] = ((d_times.weekday + 4) % 7) * fw[3]
    df['w5'] = ((d_times.weekday + 5) % 7) * fw[3]
    df['w6'] = ((d_times.weekday + 6) % 7) * fw[3]
    
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    df['log_month'] = np.log10(3+df.time/(60 * 24 * 30)) * fw[8]
    df = df.drop(['time'], axis=1)
    
    return df

In [4]:
def process_one_cell(df_train, df_test, th):    
    place_counts = df_train.place_id.value_counts()
    mask = (place_counts[df_train.place_id.values] >= th).values
    df_train = df_train.loc[mask]
    
    row_ids = df_test.row_id
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=35, weights=calculate_distance, 
                               metric='manhattan')
    clf.fit(df_train[feature_list], df_train.place_id)
    predictions = clf.predict_proba(df_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [5]:
def run_prediction(df_train, df_test):
    df_train = prepare_data(df_train)
    df_test = prepare_data(df_test)
    df_test['p1'] = np.nan
    df_test['p2'] = np.nan
    df_test['p3'] = np.nan
    
    prediction_result = process_one_cell(df_train, df_test, 5)
    prediction_result.sort_index(inplace=True)
    return prediction_result

In [6]:
def run_validation():
    df = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    n_cell_x = 10
    n_cell_y = 20
    x_length = 10 / n_cell_x
    y_length = 10 / n_cell_y
    total_score = 0
    score_count = 0
    for x_index in range(0, n_cell_x):
        start_time = time.time()
        for y_index in range(0, n_cell_y):
            min_x = x_index * x_length
            max_x = (x_index + 1) * x_length
            min_y = y_index * y_length
            max_y = (y_index + 1) * y_length
            
            # include the edge
            if(y_index + 1 == n_cell_y):
                max_y += 0.1
            if(x_index + 1 == n_cell_x):
                max_x += 0.1

            df_train_cell = df[(df.time <= 786239 * 0.875) & \
                               (df.x >= min_x - 0.1) & \
                               (df.x < max_x + 0.1) & \
                               (df.y >= min_y - 0.1) & \
                               (df.y < max_y + 0.1)].copy()
            
            df_validation_cell = df[(df.time > 786239 * 0.875) & \
                                    (df.x >= min_x) & \
                                    (df.x < max_x) & \
                                    (df.y >= min_y) & \
                                    (df.y < max_y)].copy()
            
            prediction_result = run_prediction(df_train_cell, df_validation_cell)
            
            # Calculate score
            prediction_result.sort_index(inplace=True)
            prediction_result['score'] = (prediction_result.p1 == df_validation_cell.place_id) * 1
            prediction_result['score'] += (prediction_result.p2 == df_validation_cell.place_id) * 0.5
            prediction_result['score'] += (prediction_result.p3 == df_validation_cell.place_id) * 0.33

            score = prediction_result.score.mean()
            #print('s1:', score, flush=True)
            total_score += score
            score_count += 1
            
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_index, flush = True)
    print("Final:", total_score/score_count, flush=True)


In [7]:
fw = [400, 1000, 1/10.5, 1/2.0, 1./22., 2, 9, 0.09, 4.5]

def calculate_distance(distances):
    return distances ** -1
run_validation()

def calculate_distance(distances):
    return distances ** -1.25
run_validation()

def calculate_distance(distances):
    return distances ** -1.5
run_validation()

def calculate_distance(distances):
    return distances ** -1.75
run_validation()

def calculate_distance(distances):
    return distances ** -2
run_validation()

Elapsed time overall: 404.14877820014954 seconds 0
Elapsed time overall: 439.0701789855957 seconds 1
Elapsed time overall: 476.3335499763489 seconds 2
Elapsed time overall: 446.86859822273254 seconds 3
Elapsed time overall: 448.76225066185 seconds 4
Elapsed time overall: 465.00969314575195 seconds 5
Elapsed time overall: 445.8953011035919 seconds 6
Elapsed time overall: 468.5411868095398 seconds 7
Elapsed time overall: 445.9453182220459 seconds 8
Elapsed time overall: 382.33224844932556 seconds 9
Final: 0.5459453708015992
Elapsed time overall: 419.28996753692627 seconds 0
Elapsed time overall: 449.696818113327 seconds 1
Elapsed time overall: 496.9801743030548 seconds 2
Elapsed time overall: 519.2563407421112 seconds 3
Elapsed time overall: 501.02886509895325 seconds 4
Elapsed time overall: 519.3279032707214 seconds 5
Elapsed time overall: 492.8720841407776 seconds 6
Elapsed time overall: 525.1751852035522 seconds 7
Elapsed time overall: 541.1275517940521 seconds 8
Elapsed time overall:

In [8]:
def calculate_distance(distances):
    return distances ** -2.25
run_validation()

def calculate_distance(distances):
    return distances ** -2.5
run_validation()

def calculate_distance(distances):
    return distances ** -2.75
run_validation()

def calculate_distance(distances):
    return distances ** -3
run_validation()

Elapsed time overall: 497.1114020347595 seconds 0
Elapsed time overall: 546.6718819141388 seconds 1
Elapsed time overall: 501.1982343196869 seconds 2
Elapsed time overall: 465.93317317962646 seconds 3
Elapsed time overall: 496.5372622013092 seconds 4
Elapsed time overall: 547.7518677711487 seconds 5
Elapsed time overall: 610.960563659668 seconds 6
Elapsed time overall: 675.9502401351929 seconds 7
Elapsed time overall: 544.1435039043427 seconds 8
Elapsed time overall: 405.81732296943665 seconds 9
Final: 0.5471226425988327
Elapsed time overall: 404.5746228694916 seconds 0
Elapsed time overall: 444.4449203014374 seconds 1
Elapsed time overall: 479.0670928955078 seconds 2
Elapsed time overall: 453.0306761264801 seconds 3
Elapsed time overall: 453.1224112510681 seconds 4
Elapsed time overall: 468.4928843975067 seconds 5
Elapsed time overall: 452.4869935512543 seconds 6
Elapsed time overall: 475.56787610054016 seconds 7
Elapsed time overall: 450.93289852142334 seconds 8
Elapsed time overall:

In [ ]:
# Write to file
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_validation_result.csv', index=False)

In [ ]:
# Run test
df_train = pd.read_csv('../train.csv',
                       usecols=['row_id','x','y','accuracy','time','place_id'])
df_test = pd.read_csv('../test.csv',
                       usecols=['row_id','x','y','accuracy','time'])
n_cell_x = 30
n_cell_y = 60 

df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_test = prepare_data(df_test, n_cell_x, n_cell_y)
df_test['p1'] = np.nan
df_test['p2'] = np.nan
df_test['p3'] = np.nan
prediction_result = run_prediction(df_train, df_test, 5)

prediction_result.sort_index(inplace=True)
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_result.csv', index=False)